In [ ]:
import os
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

UPLOAD_FOLDER = 'E:/Copies'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def get_content(filename):
    file_path = os.path.join("E:\Copies", filename)
    with open(file_path, "r", encoding='utf-8') as file:
        content = file.read()
    return content


def search_files(query):
    files = []
    # Iterate through files in the "Copies" folder
    for filename in os.listdir(UPLOAD_FOLDER):
        # Check if the query matches the file name
        if query.lower() in filename.lower():
            files.append(filename)
    return files

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/legal_user')
def legal_user():
    return render_template('legal_user.html')

@app.route('/document_analysis', methods=['GET', 'POST'])
def document_analysis():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        uploaded_file = request.files['file']
        if uploaded_file.filename == '':
            return redirect(request.url)
        uploaded_file.save(os.path.join(app.config['UPLOAD_FOLDER'], uploaded_file.filename))
        return redirect(url_for('document_analysis'))
    if request.method == 'GET':
        search_query = request.args.get('search', '')
        if search_query:
            files = search_files(search_query)
            return render_template('document_analysis.html', files=files, search_query=search_query)
    return render_template('document_analysis.html')

@app.route("/open_file/<filename>")
def get_file(filename):
    if filename.endswith(".txt"):
        file_contents = get_content(filename)
        return render_template("file_display.html", filename=filename, file_contents=file_contents)
    else:
        return render_template("file_display.html", filename=filename)

from summary_gen import generate_summary

@app.route("/summary/<filename>")
def get_summary(filename):
    if filename.endswith(".txt"):
        file_contents = get_content(filename)
        summary = generate_summary(file_contents)
        return render_template("summary.html", filename=filename, summary=summary)
    else:
        return render_template("summary.html", filename=filename)


import spacy

# Load the SpaCy English model with NER
nlp = spacy.load("en_core_web_sm")

@app.route("/informatics/<filename>")
def get_informatics(filename):
    document_content = get_content(filename)
    doc = nlp(document_content)
    entities = [{"text": ent.text, "label": ent.label_} for ent in doc.ents]
    return render_template("informatics.html", filename=filename, entities=entities)

if __name__ == '__main__':
    app.run(debug=True)